# Problem Formulation

We have a standard Markov Decision Process (MDP) defined by:

$$
(s_{t+1}, r_t) \sim P(s_{t+1}, r_t \mid s_t, a_t)
$$

where:

- $s_t \in S$ is the state,
- $a_t \in A$ is the action taken from the policy $\pi_\theta(a_t \mid s_t)$,
- $r_t \in R$ is the reward,
- $P(s_{t+1} \mid s_t, a_t)$ is the transition function.

We modify this MDP by introducing an augmented state representation:

$$
\tilde{s}_t = (s_t, \hat{p}_t)
$$

where 

$$
\hat{p}_t = f_\phi(s_t, a_t)
$$

is the extra Peek feature predicted by a learned dynamics model $f_\phi$.

The goal is to prove that training PPO on $\tilde{s}_t$ leads to improved policy performance.

## Effect on Policy Gradient Variance

PPO uses the advantage function:

$$
A_t = Q(s_t, a_t) - V(s_t)
$$

where:

- $Q(s_t, a_t)$ is the state-action value function,
- $V(s_t) = \mathbb{E}_{a_t \sim \pi}[Q(s_t, a_t)]$ is the value function.

PPO updates the policy by maximizing the clipped surrogate objective:

$$
L_{\text{PPO}}(\theta) = \mathbb{E}_{s,a \sim \pi} \left[ \min \left( r_t(\theta) A_t, \operatorname{clip}(r_t(\theta), 1 - \epsilon, 1 + \epsilon) A_t \right) \right]
$$

where

$$
 r_t(\theta) = \frac{\pi_\theta(a_t \mid s_t)}{\pi_{\theta_{\text{old}}}(a_t \mid s_t)}
$$

is the probability ratio.

Using the augmented state $\tilde{s}_t$, we redefine:

$$
A_t' = Q(\tilde{s}_t, a_t) - V(\tilde{s}_t)
$$

If $\hat{p}_t$ provides useful predictive information about future rewards, then:

$$
V(\tilde{s}_t) = \mathbb{E}_{a_t \sim \pi}[Q(\tilde{s}_t, a_t)]
$$

is a lower variance estimator of $Q(s_t, a_t)$, because it incorporates additional information.

### Proof by Variance Reduction

By the law of total variance, the variance of the original advantage estimate is:

$$
\operatorname{Var}[A_t] = \operatorname{Var}[Q(s_t, a_t) - V(s_t)]
$$

With the augmented state $\tilde{s}_t$:

$$
\operatorname{Var}[A_t'] = \operatorname{Var}[Q(\tilde{s}_t, a_t) - V(\tilde{s}_t)]
$$

Since $\tilde{s}_t$ includes additional predictive features, the conditional variance satisfies:

$$
\operatorname{Var}[Q(\tilde{s}_t, a_t) \mid \tilde{s}_t] \leq \operatorname{Var}[Q(s_t, a_t) \mid s_t]
$$

This implies:

$$
\operatorname{Var}[A_t'] \leq \operatorname{Var}[A_t]
$$

Since PPO’s policy gradient update depends on the expectation of $A_t$, a lower-variance estimate leads to more stable updates and improved policy convergence.

# Hoeffding’s Inequality

The Hoeffding bound states that if $X_1, X_2, \dots, X_n$ are independent and bounded random variables such that:

$$
a \leq X_i \leq b
$$

for all $i$, then for their empirical mean:

$$
\bar{X} = \frac{1}{n} \sum_{i=1}^{n} X_i
$$

the probability that $\bar{X}$ deviates from its expectation $E[\bar{X}]$ by more than $\epsilon$ is bounded by:

$$
P(\mid \bar{X} - E[\bar{X}] \mid \geq \epsilon) \leq 2 \exp \left( \frac{-2n\epsilon^2}{(b-a)^2} \right)
$$

This tells us that:

- More samples (larger $n$) reduce the probability of deviation.
- Lower variance (smaller $b-a$) leads to a tighter bound, meaning we need fewer samples for the same confidence level.

# Applying Hoeffding’s Bound to PPO Advantage Estimation

In PPO, we estimate the advantage function:

$$
A_t = Q(s_t, a_t) - V(s_t)
$$

where:

- $Q(s_t, a_t)$ is the state-action value function.
- $V(s_t)$ is the value function.

Since PPO updates the policy based on the advantage function $A_t$, accurate estimation of $A_t$ is crucial for stable training.

In practice, $A_t$ is estimated using Monte Carlo rollouts or Generalized Advantage Estimation (GAE), which involves averaging multiple samples:

$$
\hat{A} = \frac{1}{n} \sum_{i=1}^{n} A_i
$$

By Hoeffding’s bound:

$$
P(\mid \hat{A} - E[\hat{A}] \mid \geq \epsilon) \leq 2 \exp \left( \frac{-2n\epsilon^2}{\sigma_A^2} \right)
$$

where $\sigma_A^2$ is the variance of the advantage estimates.

### Key Insight: Reducing $\sigma_A^2$ Lowers Sample Complexity

- If the variance of advantage estimates $\sigma_A^2$ is high, then we need more samples $n$ to achieve a desired error bound $\epsilon$.
- If the variance $\sigma_A^2$ is low, we need fewer samples to reach the same confidence level.

Thus, reducing variance in $A_t$ accelerates PPO training by decreasing the required number of interactions with the environment.

# Effect of Extra Features on Variance Reduction

Now, we analyze how adding extra features $\hat{e}_t = f_\phi(s_t, a_t)$ impacts variance.

The new augmented state representation:

$$
\tilde{s}_t = (s_t, \hat{e}_t)
$$

leads to a better advantage estimate:

$$
A_t' = Q(\tilde{s}_t, a_t) - V(\tilde{s}_t)
$$

If $\hat{e}_t$ contains useful predictive information, it helps in reducing the error of $V(s_t)$:

$$
\operatorname{Var}[V(\tilde{s}_t)] \leq \operatorname{Var}[V(s_t)]
$$

Since $A_t'$ is derived from $Q(\tilde{s}_t, a_t) - V(\tilde{s}_t)$, the variance of the advantage function also decreases:

$$
\operatorname{Var}[A_t'] \leq \operatorname{Var}[A_t]
$$

By Hoeffding’s bound, reducing $\operatorname{Var}[A_t]$ directly reduces the number of samples needed for a given confidence level.

Thus, using extra features in the observation space improves PPO’s sample efficiency, making training faster and more stable.




## Sample Efficiency and Convergence Rate

In reinforcement learning, sample efficiency is often analyzed using policy improvement guarantees.

Define $\pi^*$ as the optimal policy and let $\pi^{(k)}$ be the policy at iteration $k$. PPO ensures monotonic improvement in expected return:

$$
J(\pi^{(k+1)}) \geq J(\pi^{(k)})
$$

However, faster improvement depends on how well the advantage function is estimated.

From our previous variance reduction proof:

$$
\operatorname{Var}[A_t'] \leq \operatorname{Var}[A_t]
$$

which implies that policy updates are less noisy, leading to faster improvement in policy performance.


## Effect on Value Function Approximation

PPO also trains a value function $V_\theta(s_t)$ by minimizing the squared Bellman error:

$$
L_{\text{VF}}(\theta) = \mathbb{E}_{s_t} \left[ (V_\theta(s_t) - R_t)^2 \right]
$$

where $R_t$ is the return.

With the augmented state, the loss function becomes:

$$
L_{\text{VF}}'(\theta) = \mathbb{E}_{\tilde{s}_t} \left[ (V_\theta(\tilde{s}_t) - R_t)^2 \right]
$$

Since $\tilde{s}_t$ includes the predicted feature $\hat{e}_t$, it provides more informative state representations. If $\hat{p}_t$ correlates with long-term returns, then:

$$
\mathbb{E} \left[ (V_\theta(\tilde{s}_t) - R_t)^2 \right] \leq \mathbb{E} \left[ (V_\theta(s_t) - R_t)^2 \right]
$$

which means the value function has lower approximation error.

By improving $V(s_t)$, the advantage estimates become more accurate, leading to better PPO updates.

### Need to do
1. Mutual Information or Granger test to test for correlation between long-term reward and extra Peek feature
2. Prove dynamics model decrease the variance of advantage function
3. Why suboptimal action still improves the model?